# Homography


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import cv2


### 1. warm up

Load the 'cereals_query' and 'cereals_frame' images.

Calculate SIFT keypoints, and try to find best matches. Use these matches to estimate the Homography parameters (using RANSAC). Mark the transformed box of the query image on top of the 'cereals_frame' image.

useful links:

https://www.learnopencv.com/homography-examples-using-opencv-python-c/

http://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_feature2d/py_feature_homography/py_feature_homography.html#feature-homography


In [2]:
img = cv2.imread('cereals_frame.jpg', 0)
img_c = cv2.imread('cereals_frame.jpg')
q = cv2.imread('cereals_query.jpg', 0)

sift = cv2.xfeatures2d.SIFT_create(nfeatures = 0, sigma=1.6, edgeThreshold=10, contrastThreshold=0.04)

kpts_img, des_img = sift.detectAndCompute(img,None)
img_kpts_output = cv2.drawKeypoints(img, kpts_img, outImage=None, flags=4)
kpts_q, des_q = sift.detectAndCompute(q,None)
q_kpts_output = cv2.drawKeypoints(q, kpts_q, outImage=None, flags=4)

bfm = cv2.BFMatcher()

# for knn match
knnm1 = bfm.knnMatch(des_img,des_q,k=2)
knn_img_to_q = bfm.knnMatch(des_q, des_img, k=2)

# max 20 matches
max_matches = 20

# for knn match
matches_img_q = list(filter(lambda m : m[0].distance < 0.4 * m[1].distance, knnm1))[:max_matches]

good_matches = [m[0] for m in knn_img_to_q if m[0].distance < 0.4 * m[1].distance]
#matches_img_q1 = sorted(bfm.match(des_img,des_q1),key=lambda x:x.distance)



#for knn match
matches_img_q_res = cv2.drawMatchesKnn(img,kpts_img,q,kpts_q,matches_img_q,outImg=None,
                          flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)
                        

#matches_img_q1_res = cv2.drawMatches(img,kpts_img,q1,kpts_q1,matches_img_q1[:20],outImg=None,
#                                     flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)

cv2.imshow('img', matches_img_q_res)
key_pressed = False
while not key_pressed:
    k = cv2.waitKey(33)
    if k != -1:  # if no key was pressed, -1 is returned
        key_pressed = True
cv2.destroyWindow('img')

if len(good_matches)>10:
    src_pts = np.float32([ kpts_q[m.queryIdx].pt for m in good_matches ]).reshape(-1,1,2)
    dst_pts = np.float32([ kpts_img[m.trainIdx].pt for m in good_matches ]).reshape(-1,1,2)
    H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
    h,w = q.shape
    pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
    dst = cv2.perspectiveTransform(pts,H)
    img_c_with_border = cv2.polylines(img_c,[np.int32(dst)],True,255,3, cv2.LINE_AA)

cv2.imshow('img', img_c_with_border)
key_pressed = False
while not key_pressed:
    k = cv2.waitKey(33)
    if k != -1:  # if no key was pressed, -1 is returned
        key_pressed = True
cv2.destroyWindow('img')

### 2. lets make a movie!

Load each and every frame in the 'cereals_movie' file, and repeat the same analysis as before. For each input frame, generate an output frame with the transormed box of the query image on top (check that you have enough 'good matched' before). Generate an output movie from all the output frames. 

Submit both the `.ipynb` notebook and your movie file.

In case the movie frames are too big, you can rescale them to a lower resolution, before performing your analysis. 


In [3]:
def recognize_frame(img, q):
    
    sift = cv2.xfeatures2d.SIFT_create(nfeatures = 0, sigma=1.6, edgeThreshold=10, contrastThreshold=0.04)

    kpts_img, des_img = sift.detectAndCompute(img,None)
    kpts_q, des_q = sift.detectAndCompute(q,None)

    bfm = cv2.BFMatcher()

    # for knn match
    knn_img_to_q = bfm.knnMatch(des_q, des_img, k=2)

    # max 20 matches
    max_matches = 20


    good_matches = [m[0] for m in knn_img_to_q if m[0].distance < 0.4 * m[1].distance][:max_matches]
    #matches_img_q1 = sorted(bfm.match(des_img,des_q1),key=lambda x:x.distance)
    if len(good_matches) > 10:
        src_pts = np.float32([ kpts_q[m.queryIdx].pt for m in good_matches ]).reshape(-1,1,2)
        dst_pts = np.float32([ kpts_img[m.trainIdx].pt for m in good_matches ]).reshape(-1,1,2)
        H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
        h,w = q.shape
        pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
        dst = cv2.perspectiveTransform(pts,H)
        img = cv2.polylines(img,[np.int32(dst)],True,255,3, cv2.LINE_AA)
    
          
    return img


# play and save a movie
video_attr = ('cereals_movie','mp4v', 20.0, 540, 960, '.mp4')
cap = cv2.VideoCapture('cereals_movie.mp4')
fourcc = cv2.VideoWriter_fourcc(*video_attr[1])
video_name = '{0}_{1}_{2}_{3}_{4}_output{5}'.format(*video_attr)
out = cv2.VideoWriter(video_name, fourcc, video_attr[2], (video_attr[3],video_attr[4]))
q = cv2.imread('cereals_query.jpg', 0)

grayscale = False

cv2.imshow('query', q)
key_pressed = False
while not key_pressed:
    k = cv2.waitKey(33)
    if k != -1:  # if no key was pressed, -1 is returned
        key_pressed = True
cv2.destroyWindow('query')

while(cap.isOpened()):
    ret, frame = cap.read()
    if ret==False:
        break
    if grayscale:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
    frame = recognize_frame(frame, q)
    cv2.imshow('movie',frame)
    out.write(frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cap.release()
out.release()
cv2.destroyAllWindows()

AttributeError: module 'cv2.cv2' has no attribute 'xfeatures2d'

## Good Luck!